<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Presidential-Election-Data" data-toc-modified-id="Presidential-Election-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Presidential Election Data</a></span></li></ul></div>

# Presidential Election Data
Determine wheather a state is regarded as conservative, liberal or swing.

In [1]:
# import necessary packages
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from io import StringIO
import numpy as np

In [2]:
# 1996 ELECTORAL AND POPULAR VOTE SUMMARY is in html format
def simple_get(url):
    """
    Attempts to get the content at 'url' by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                # r.text is the content of the response in unicode, 
                # and r.content is the content of the response in bytes.
                return resp.content
            else:
                return None
            
    except RequestException as e:
        log_error('Error during requests to {0}:{1}'.format(url, str(e)))
        return None

In [3]:
def is_good_response(resp):
    """
    Return True if the response seems to be HTML/HTM, Flase otherwise.
    """
    content_type = resp.headers['Content-Type']
    return (resp.status_code == 200 
            and content_type is not None
            and content_type.find('html') > -1)

In [4]:
def log_error(e):
    """
    Print log errors.
    """
    print(e)

In [5]:
# Download 1996 presidential ELECTORAL AND POPULAR VOTE 
url = 'https://transition.fec.gov/pubrec/fe1996/elecpop.htm'
response = simple_get(url)
if response is not None:
    htm = BeautifulSoup(response, 'html.parser')
    # cast to string
    para = str(htm.find_all('pre'))
    temp_content = para[para.find('>AL'):]
    table_content = temp_content[1:temp_content.find('<st')]

In [6]:
table_content_li = [x for x in table_content.split('\r\n')]

In [7]:
content = []
for row in table_content_li[:-1]:
    a = row.split('        ')
    if a[1] == '':
        a[1] = 'n'
    if a[2] == '':
        a[2] = 'n'
    content.append(a)

In [8]:
# process exception: different length of row
content[8]

['DC', '3', 'n', '     158,220       17,339', '3,611', '      185,726 ']

In [9]:
update = content[8][3] + ' ' + content[8][4]
content[8][3] = update

In [10]:
temp_row = content[8]
del temp_row[4]
temp_row

['DC', '3', 'n', '     158,220       17,339 3,611', '      185,726 ']

In [16]:
# replace exception with updated row
content[8] = temp_row

In [17]:
# convert list to numpy array then to dataframe
df96 = pd.DataFrame(content)

In [20]:
df96.columns = ['State', 'Clinton','Dole','Popular vote','Total Popular vote']

In [21]:
df96

,State,Clinton,Dole,Popular vote,Total Popular vote
0,AL,n,9,"662,165 769,044 92,149","1,534,349"
1,AK,n,3,"80,380 122,746 26,333","241,620"
2,AZ,8,n,"653,288 622,073 112,072","1,404,405"
3,AR,6,n,"475,171 325,416 69,884","884,262"
4,CA,54,n,"5,119,835 3,828,380 697,847","10,019,484"
5,CO,n,8,"671,152 691,848 99,629","1,510,704"
6,CT,8,n,"735,740 483,109 139,523","1,392,614"
7,DE,3,n,"140,355 99,062 28,719","270,845"
8,DC,3,n,"158,220 17,339 3,611","185,726"
9,FL,25,n,"2,546,870 2,244,536 483,870","5,303,794"
